In [1]:
import pandas as pd
import re
from urllib.parse import urlparse

In [25]:
# Read the file
df_Genre = pd.read_csv('/Users/daan/Desktop/HVA/Database management/FinalSales1.csv', sep=',', encoding='ISO-8859-1')

# Fill NaN values in 'genre' with a placeholder genre
df_Genre['genre'] = df_Genre['genre'].fillna('Unknown')

# Re-create the genre_id mapping after filling NaNs
unique_genres = df_Genre['genre'].unique()
genre_id_mapping = {genre: idx + 1 for idx, genre in enumerate(unique_genres)}

# Map genres to IDs
df_Genre['genre_id'] = df_Genre['genre'].map(genre_id_mapping)

# View the updated dataframe with 'genre' and 'genre_id'
print(df_Genre[['genre', 'genre_id']].head())



     genre  genre_id
0    Drama         1
1  Unknown         2
2   Comedy         3
3  Unknown         2
4    Drama         1


In [28]:
import pandas as pd

# Read the file
df_Genre = pd.read_csv('/Users/daan/Desktop/HVA/Database management/FinalSales1.csv', sep=',', encoding='ISO-8859-1')

# Fill NaN values in 'genre' with a placeholder genre
df_Genre['genre'] = df_Genre['genre'].fillna('Unknown')

# Re-create the genre_id mapping after filling NaNs
unique_genres = df_Genre['genre'].unique()
genre_id_mapping = {genre: idx + 1 for idx, genre in enumerate(unique_genres)}

# Map genres to IDs
df_Genre['genre_id'] = df_Genre['genre'].map(genre_id_mapping)

# Ensure 'Movie_ID' is included and select relevant columns
df_Genre = df_Genre[['genre_id', 'genre','Movie_ID']]

# Print the updated DataFrame
print(df_Genre.head())


   genre_id    genre  Movie_ID
0         1    Drama         1
1         2  Unknown         3
2         3   Comedy     17328
3         2  Unknown     17329
4         1    Drama     17330


In [29]:
import psycopg2
from psycopg2 import sql

In [31]:
import psycopg2
import pandas as pd

# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname='DBM',
    user='postgres',
    password='Dk141621?18!',
    host='localhost',
    port='5432'
)

# Create a cursor object
cursor = conn.cursor()

# Define the table creation query
create_table_query = """
CREATE TABLE IF NOT EXISTS Genre (
    Genre_ID INTEGER PRIMARY KEY,
    genre TEXT NOT NULL,
    Movie_ID INTEGER NOT NULL
);
"""

# Execute the table creation query
cursor.execute(create_table_query)

# Commit the changes
conn.commit()

# Define the insert query
insert_query = """
INSERT INTO expert_reviews (Genre_ID, genre, Movie_ID)
VALUES (%s, %s, %s)
ON CONFLICT (Genre_ID) DO UPDATE
SET genre = EXCLUDED.genre,
    Movie_ID = EXCLUDED.Movie_ID;
"""

# Read your dataframe (assuming you have it as df_Genre)
df_Genre = pd.read_csv('/Users/daan/Desktop/HVA/Database management/FinalSales1.csv', sep=',', encoding='ISO-8859-1')
df_Genre['genre'] = df_Genre['genre'].fillna('Unknown')
unique_genres = df_Genre['genre'].unique()
genre_id_mapping = {genre: idx + 1 for idx, genre in enumerate(unique_genres)}
df_Genre['genre_id'] = df_Genre['genre'].map(genre_id_mapping)

# Insert data into the table
for index, row in df_Genre.iterrows():
    cursor.execute(insert_query, (row['genre_id'], row['genre'], row['Movie_ID']))

# Commit the changes
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()
